In [74]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from geopy.distance import geodesic
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
E_Net = ElasticNet(alpha=0.1, l1_ratio=.5, random_state=42, tol=0.1)
LG = LinearRegression()
RF_Regressor = RandomForestRegressor()
GBR = GradientBoostingRegressor()
tree_reg = DecisionTreeRegressor(random_state=42)

In [3]:
data = pd.read_pickle('Regression_data.pkl')
data = data.loc[((data['Number of rooms']>0) & 
                 (data['Number of rooms']<=9) &
                 (data['Transaction cost']<=2e6) &
                 (data['Transaction cost']>=6e4))]
data.head(5)

,Transaction date,Transaction cost,Postcode,Type of property,Built surface,Number of rooms,Ground surface,coord
0,8.0,251500.0,1310.0,1.0,147.0,5.0,1501.0,"(46.247576, 5.119191000000001)"
2,8.0,174500.0,1000.0,0.0,80.0,2.0,0.0,"(46.204017, 5.218208)"
4,1.0,157500.0,1440.0,1.0,103.0,4.0,1569.0,"(46.236276000000004, 5.2627559999999995)"
7,6.0,90000.0,1000.0,0.0,61.0,2.0,0.0,"(46.200396999999995, 5.219821)"
11,13.0,95000.0,1000.0,0.0,58.0,3.0,0.0,"(46.204702000000005, 5.210812000000001)"


In [11]:
def pricePrediction(obj, model):
    data_selected = data_train[(data_train['Postcode'] > (obj['Postcode'] - 2)) 
                & (data_train['Postcode'] < (obj['Postcode'] + 2)) 
                & (data_train['Type of property'] == obj['Type of property'])]
    coord = obj.coord

    def calc_distance(x):
        site1_coords = coord
        site2_coords = x.coord
        return geodesic(site1_coords, site2_coords).km

    data_selected['distance'] = data_selected.apply(calc_distance, axis=1)
    data_selected = data_selected.sort_values(by="distance").head(150)

    X = data_selected[['Transaction date','Built surface','Ground surface','Number of rooms']]
    y = data_selected['Transaction cost']

    model.fit(X, y)
    input = obj[['Transaction date','Built surface','Ground surface','Number of rooms']]
    input = np.array(input).reshape(1, -1)
    price_prediction = model.predict(input)

    return price_prediction[0]

In [61]:
data_test = data.sample(frac = 0.0003)
  
data_train = data.drop(data_test.index)

In [62]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1771 entries, 1440669 to 2013694
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction date  1771 non-null   float64
 1   Transaction cost  1771 non-null   float64
 2   Postcode          1771 non-null   float64
 3   Type of property  1771 non-null   float64
 4   Built surface     1771 non-null   float64
 5   Number of rooms   1771 non-null   float64
 6   Ground surface    1771 non-null   float64
 7   coord             1771 non-null   object 
dtypes: float64(7), object(1)
memory usage: 124.5+ KB


# Test final 

## Random forest

In [63]:
data_test['Transaction cost prediction'] = data_test.apply(lambda x : pricePrediction(x, RF_Regressor), axis=1)
data_test['Prediction error'] = data_test.apply(lambda x:
                            (np.abs(x['Transaction cost'] - x['Transaction cost prediction'])),axis=1)
data_test['Relative error'] = data_test.apply(lambda x:
                            (x['Prediction error'] / x['Transaction cost']),axis=1)

In [64]:
data_test.describe()

,Transaction date,Transaction cost,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Transaction cost prediction,Prediction error,Relative error
count,1771.000000,1.771000e+03,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1.771000e+03,1.771000e+03,1771.000000
mean,1356.889328,2.274027e+05,52990.111801,0.557877,80.226426,3.511011,399.551666,2.252294e+05,4.400409e+04,0.238181
std,687.306291,1.881854e+05,27052.168122,0.496779,36.950939,1.461470,577.703110,1.668414e+05,6.822407e+04,0.377650
min,1.000000,6.000000e+04,1170.000000,0.000000,10.000000,1.000000,0.000000,6.166700e+04,0.000000e+00,0.000000
25%,799.500000,1.200000e+05,32430.000000,0.000000,53.000000,2.000000,0.000000,1.276257e+05,9.049800e+03,0.053339
50%,1386.000000,1.700000e+05,54220.000000,1.000000,77.000000,4.000000,203.000000,1.733538e+05,2.400080e+04,0.138685
75%,1934.000000,2.631850e+05,76620.000000,1.000000,103.000000,4.000000,554.000000,2.611771e+05,5.033790e+04,0.286354
max,2555.000000,1.540000e+06,97460.000000,1.000000,190.000000,8.000000,4236.000000,1.448500e+06,1.022790e+06,5.581159


## Gradient boosting regression

In [65]:
data_test['Transaction cost prediction'] = data_test.apply(lambda x : pricePrediction(x, GBR), axis=1)
data_test['Prediction error'] = data_test.apply(lambda x:
                            (np.abs(x['Transaction cost'] - x['Transaction cost prediction'])),axis=1)
data_test['Relative error'] = data_test.apply(lambda x:
                            (x['Prediction error'] / x['Transaction cost']),axis=1)

In [66]:
data_test.describe()

,Transaction date,Transaction cost,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Transaction cost prediction,Prediction error,Relative error
count,1771.000000,1.771000e+03,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1.771000e+03,1771.000000,1771.000000
mean,1356.889328,2.274027e+05,52990.111801,0.557877,80.226426,3.511011,399.551666,2.264469e+05,43603.025766,0.238418
std,687.306291,1.881854e+05,27052.168122,0.496779,36.950939,1.461470,577.703110,1.732588e+05,67098.613599,0.403613
min,1.000000,6.000000e+04,1170.000000,0.000000,10.000000,1.000000,0.000000,5.481524e+04,0.000000,0.000000
25%,799.500000,1.200000e+05,32430.000000,0.000000,53.000000,2.000000,0.000000,1.256032e+05,9209.851318,0.051773
50%,1386.000000,1.700000e+05,54220.000000,1.000000,77.000000,4.000000,203.000000,1.727846e+05,22868.252926,0.131954
75%,1934.000000,2.631850e+05,76620.000000,1.000000,103.000000,4.000000,554.000000,2.600627e+05,51050.245884,0.286189
max,2555.000000,1.540000e+06,97460.000000,1.000000,190.000000,8.000000,4236.000000,1.485598e+06,821677.140387,6.568242


## Decision tree regression

In [35]:
data_test['Transaction cost prediction'] = data_test.apply(lambda x : pricePrediction(x, tree_reg), axis=1)
data_test['Prediction error'] = data_test.apply(lambda x:
                            (np.abs(x['Transaction cost'] - x['Transaction cost prediction'])),axis=1)
data_test['Relative error'] = data_test.apply(lambda x:
                            (x['Prediction error'] / x['Transaction cost']),axis=1)

In [36]:
data_test.describe()

,Transaction date,Transaction cost,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Transaction cost prediction,Prediction error,Relative error
count,590.000000,5.900000e+02,590.000000,590.000000,590.000000,590.000000,590.000000,5.900000e+02,5.900000e+02,590.000000
mean,1319.377966,2.292984e+05,53522.245763,0.559322,81.825424,3.547458,389.093220,2.230652e+05,5.631549e+04,0.269108
std,698.777665,2.024258e+05,27260.069294,0.496890,37.296386,1.486882,558.221399,1.815758e+05,9.937282e+04,0.408503
min,15.000000,6.000000e+04,1130.000000,0.000000,14.000000,1.000000,0.000000,6.000000e+04,0.000000e+00,0.000000
25%,744.250000,1.160500e+05,31417.500000,0.000000,55.250000,3.000000,0.000000,1.191250e+05,5.000000e+03,0.035594
50%,1359.000000,1.670000e+05,57460.000000,1.000000,79.000000,4.000000,199.000000,1.700000e+05,2.500000e+04,0.156011
75%,1884.750000,2.700000e+05,77530.000000,1.000000,103.000000,5.000000,592.000000,2.600000e+05,6.326250e+04,0.341727
max,2554.000000,1.600000e+06,97490.000000,1.000000,191.000000,9.000000,4090.000000,1.500000e+06,1.103400e+06,3.868182


## Linear regression

In [37]:
data_test['Transaction cost prediction'] = data_test.apply(lambda x : pricePrediction(x, LG), axis=1)
data_test['Prediction error'] = data_test.apply(lambda x:
                            (np.abs(x['Transaction cost'] - x['Transaction cost prediction'])),axis=1)
data_test['Relative error'] = data_test.apply(lambda x:
                            (x['Prediction error'] / x['Transaction cost']),axis=1)

In [38]:
data_test.describe()

,Transaction date,Transaction cost,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Transaction cost prediction,Prediction error,Relative error
count,590.000000,5.900000e+02,590.000000,590.000000,590.000000,590.000000,590.000000,5.900000e+02,5.900000e+02,590.000000
mean,1319.377966,2.292984e+05,53522.245763,0.559322,81.825424,3.547458,389.093220,2.309537e+05,5.687994e+04,0.306868
std,698.777665,2.024258e+05,27260.069294,0.496890,37.296386,1.486882,558.221399,1.687972e+05,8.128617e+04,0.418995
min,15.000000,6.000000e+04,1130.000000,0.000000,14.000000,1.000000,0.000000,-7.035387e+04,1.925847e+01,0.000108
25%,744.250000,1.160500e+05,31417.500000,0.000000,55.250000,3.000000,0.000000,1.333662e+05,1.610849e+04,0.085790
50%,1359.000000,1.670000e+05,57460.000000,1.000000,79.000000,4.000000,199.000000,1.791020e+05,3.326636e+04,0.190107
75%,1884.750000,2.700000e+05,77530.000000,1.000000,103.000000,5.000000,592.000000,2.682741e+05,6.930032e+04,0.355468
max,2554.000000,1.600000e+06,97490.000000,1.000000,191.000000,9.000000,4090.000000,1.452202e+06,1.042620e+06,4.605916


## Elastic net regression

In [39]:
data_test['Transaction cost prediction'] = data_test.apply(lambda x : pricePrediction(x, E_Net), axis=1)
data_test['Prediction error'] = data_test.apply(lambda x:
                            (np.abs(x['Transaction cost'] - x['Transaction cost prediction'])),axis=1)
data_test['Relative error'] = data_test.apply(lambda x:
                            (x['Prediction error'] / x['Transaction cost']),axis=1)

In [40]:
data_test.describe()

,Transaction date,Transaction cost,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Transaction cost prediction,Prediction error,Relative error
count,590.000000,5.900000e+02,590.000000,590.000000,590.000000,590.000000,590.000000,5.900000e+02,5.900000e+02,590.000000
mean,1319.377966,2.292984e+05,53522.245763,0.559322,81.825424,3.547458,389.093220,2.311584e+05,5.672825e+04,0.306650
std,698.777665,2.024258e+05,27260.069294,0.496890,37.296386,1.486882,558.221399,1.691334e+05,8.114010e+04,0.419906
min,15.000000,6.000000e+04,1130.000000,0.000000,14.000000,1.000000,0.000000,-8.576117e+04,3.317892e+00,0.000020
25%,744.250000,1.160500e+05,31417.500000,0.000000,55.250000,3.000000,0.000000,1.341549e+05,1.560915e+04,0.082519
50%,1359.000000,1.670000e+05,57460.000000,1.000000,79.000000,4.000000,199.000000,1.791774e+05,3.281841e+04,0.190017
75%,1884.750000,2.700000e+05,77530.000000,1.000000,103.000000,5.000000,592.000000,2.686875e+05,6.827833e+04,0.357868
max,2554.000000,1.600000e+06,97490.000000,1.000000,191.000000,9.000000,4090.000000,1.475017e+06,1.044644e+06,4.606037


## While?

In [71]:
def estimator(obj, model):
    data_geo = data_train[(data_train['Postcode'] >(obj["Postcode"]-2))
                  &(data_train['Postcode'] <(obj["Postcode"]+2))
                  &(data_train['Type of property'] == obj['Type of property'])]
    data_geo=data_geo.assign(distance=2)
    data_geo['distance'] = data_geo.apply(calc_distance, axis = 1)
    
    n=20
    min_rmse = 1000000000
    estimation = 0
    
    while n<900:
        data_geo_reg=data_geo.sort_values(by="distance").head(n)
        X = data_geo_reg[['Transaction date','Built surface','Ground surface', 'Number of rooms']]
        y = data_geo_reg['Transaction cost']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=9)
        
        model.fit(X_train, y_train) 
        pred = model.predict(X_test)
        test_set_rmse = (np.sqrt(mean_squared_error(y_test, pred))) 
        if test_set_rmse < min_rmse:
            min_rmse=test_set_rmse
            input = obj[['Transaction date','Built surface','Ground surface','Number of rooms']]
            input = np.array(input).reshape(1, -1)
            estimation = model.predict(input)
            n_min=n
        n = n + 30
    return estimation[0]

In [75]:
data_test['Transaction cost prediction'] = data_test.apply(lambda x : estimator(x, RF_Regressor), axis=1)
data_test['Prediction error'] = data_test.apply(lambda x:
                            (np.abs(x['Transaction cost'] - x['Transaction cost prediction'])),axis=1)
data_test['Relative error'] = data_test.apply(lambda x:
                            (x['Prediction error'] / x['Transaction cost']),axis=1)

140
[91825.]
80
[1212540.]
830
[103632.6538]
50
[188773.1222]
140
[376600.]
20
[241000.]
20
[237670.]
20
[199654.]
80
[182697.2]
230
[331864.3]
20
[213470.]
80
[152000.]
650
[171376.78]
20
[611861.]
20
[244674.]
20
[113439.4]
260
[130868.35]
20
[160415.22]
50
[108466.]
20
[747600.]
50
[295861.2]
20
[136702.]
50
[149251.5]
20
[182112.]
80
[178743.]
50
[466530.]
50
[217239.]
20
[674820.9]
140
[309907.5]
200
[70038.7]
50
[401706.]
20
[164028.874]
350
[144891.7]
230
[186240.]
110
[130320.]
20
[226840.5]
20
[253915.]
110
[93674.]
290
[144534.84]
140
[108754.31]
80
[264714.]
290
[105216.2]
20
[969498.]
20
[123129.]
140
[118171.04]
230
[106652.]
50
[175041.3]
470
[139133.44]
50
[208396.7]
20
[250830.]
20
[84040.]
50
[98550.]
50
[423751.3356]
140
[297648.]
50
[672270.4]
20
[152119.5]
110
[98785.8]
50
[135433.]
170
[212737.6]
20
[96964.34]
20
[117015.]
20
[118227.31]
230
[250987.49]
320
[130530.]
80
[898935.4]
20
[292244.]
20
[135370.]
20
[91652.01]
140
[259177.04]
350
[132793.46]
20
[147580.]


In [80]:
data_test.describe()

,Transaction date,Transaction cost,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Transaction cost prediction,Prediction error,Relative error
count,1771.000000,1.771000e+03,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1.771000e+03,1771.000000,1771.000000
mean,1356.889328,2.274027e+05,52990.111801,0.557877,80.226426,3.511011,399.551666,2.234710e+05,44074.025137,0.229172
std,687.306291,1.881854e+05,27052.168122,0.496779,36.950939,1.461470,577.703110,1.689444e+05,66119.011249,0.333025
min,1.000000,6.000000e+04,1170.000000,0.000000,10.000000,1.000000,0.000000,6.235500e+04,0.000000,0.000000
25%,799.500000,1.200000e+05,32430.000000,0.000000,53.000000,2.000000,0.000000,1.260711e+05,9420.950000,0.055810
50%,1386.000000,1.700000e+05,54220.000000,1.000000,77.000000,4.000000,203.000000,1.731500e+05,23400.000000,0.135378
75%,1934.000000,2.631850e+05,76620.000000,1.000000,103.000000,4.000000,554.000000,2.580691e+05,54476.000000,0.291526
max,2555.000000,1.540000e+06,97460.000000,1.000000,190.000000,8.000000,4236.000000,1.448500e+06,889964.300000,4.438496
